In [ ]:
# Packages used
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pymysql as mysql

import warnings
warnings.filterwarnings('ignore')

## Import CSV files

## Data Exploration and Pre-processing

## Transformations

## Connect to MySQL Server

In [ ]:
userName = ''
userPass = ''
conn = mysql.Connect(
    host = 'localhost', 
    port = int(3306), 
    user = userName, 
    passwd = userPass, 
    db = 'SAKILA'
)